# Assignment 2
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.out** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **run your notebook and keep all running logs** so that we can check.

## 1 Data

We will conduct experiments on Conll2003, which contains 14041 sentences, and each sentence is annotated with the corresponding named entity tags. You can download the dataset from the following
link: https://data.deepai.org/conll2003.zip. We only focus on the token and NER tags, which are
the first and last columns in the dataset. The dataset is in the IOB format, which is a common format
for named entity recognition. The IOB format 1 is a simple way to represent the named entity tags. For
example, the sentence “I went to New York City last week” is annotated as follows:

    I O
    went O
    to O
    New B-LOC
    York I-LOC
    City I-LOC
    last O
    week O

In [36]:
def load_data():
    # load data
    import os
    import numpy as np

    data_dir = os.path.join(os.getcwd(), "data")
    train_path = os.path.join(data_dir, 'train.txt')
    valid_path = os.path.join(data_dir, 'valid.txt')
    test_path  = os.path.join(data_dir, 'test.txt')

    with open(train_path, 'r', encoding="utf-8") as f:
        train_raw = [l.strip() for l in f.readlines()]
        train = list()
        start_of_sentence = 0
        for i in range(len(train_raw)):
            if train_raw[i] == '': # end of sentence
                sent = list()
                tags = list()
                for l in train_raw[start_of_sentence:i]:
                    l = l.split(' ')
                    sent.append(l[0])
                    tags.append(l[-1])
                train.append((sent, tags))
                start_of_sentence = i+1

    with open(valid_path, 'r', encoding="utf-8") as f:
        valid_raw = [l.strip() for l in f.readlines()]
        valid = list()
        start_of_sentence = 0
        for i in range(len(valid_raw)):
            if valid_raw[i] == '': # end of sentence
                sent = list()
                tags = list()
                for l in valid_raw[start_of_sentence:i]:
                    l = l.split(' ')
                    sent.append(l[0])
                    tags.append(l[-1])
                valid.append((sent, tags))
                start_of_sentence = i+1
    with open(test_path, 'r', encoding="utf-8") as f:
        test_raw = [l.strip() for l in f.readlines()]
        test = list()
        start_of_sentence = 0
        for i in range(len(test_raw)):
            if test_raw[i] == '': # end of sentence
                sent = list()
                tags = list()
                for l in test_raw[start_of_sentence:i]:
                    l = l.split(' ')
                    sent.append(l[0])
                    tags.append(l[-1])
                test.append((sent, tags))
                start_of_sentence = i+1
    
    return train, valid, test

In [37]:
train, valid, test = load_data()
# print(train)
# print(valid)
# print(test)

## 2 Tagger
    You will train your tagger on the train set and evaluate it on the dev set. And then, you may tune the
    hyperparameters of your tagger to get the best performance on the dev set. Finally, you will evaluate
    your tagger on the test set to get the final performance.

    https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)

    There are some key points you should pay attention to:
    • You will batch the sentences in the dataset to accelerate the training process. To batch the sentences, you may need to pad the sentences to the same length.
    • You are free to design the model architecture with (Bi)LSTM or Transformer unit for each part, but please do not use any pretrained weights in your basic taggers.
    • You will adjust the hyperparameters of your tagger to get the best performance on the dev set. The hyperparameters include the learning rate, batch size, the number of hidden units, the number of
    layers, the dropout rate, etc.
    • You will use seqeval to evaluate your tagger on the dev set and the test set.


### 2.1 LSTM Tagger
    We will first use an LSTM tagger to solve the NER problem. There is a very simple implementation of the
    LSTM tagger on PyTorch website https://pytorch.org/tutorials/beginner/nlp/sequence_models_
    tutorial.html. You can refer to this implementation to implement your LSTM tagger.


In [11]:
# Author: Robert Guthrie

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [12]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward0>))


In [13]:
# prepare the data
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

# Tags are: DET - determiner; NN - noun; V - verb
# For example, the word "The" is a determiner
training_data = train
word_to_ix = {}
tag_to_ix = {}  # Assign each tag with a unique index
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
    for word in tags:
        if word not in tag_to_ix:  # word has not been assigned an index yet
            tag_to_ix[word] = len(tag_to_ix)  # Assign each word with a unique index

# print(word_to_ix)
print(tag_to_ix)

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 64
HIDDEN_DIM = 64

{'O': 0, 'B-ORG': 1, 'B-MISC': 2, 'B-PER': 3, 'I-PER': 4, 'B-LOC': 5, 'I-ORG': 6, 'I-MISC': 7, 'I-LOC': 8}


In [14]:
# create the model
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [32]:
# train the model
LSTM_model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(LSTM_model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[1][0], word_to_ix)
    tag_scores = LSTM_model(inputs)
    # print(tag_scores)

training_losses = []
for epoch in range(30):  # again, normally you would NOT do 300 epochs, it is toy data
    training_loss = 0.0
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        LSTM_model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = LSTM_model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        training_loss += loss.item()

        loss.backward()
        optimizer.step()
    training_losses.append(training_loss)
    print(f'epoch: {epoch}, training_loss {training_loss}')

epoch: 0, training_loss 8289.374361423128
epoch: 1, training_loss 4861.855665402698
epoch: 2, training_loss 3578.678978038055
epoch: 3, training_loss 2797.6594345227795
epoch: 4, training_loss 2255.451435390626
epoch: 5, training_loss 1852.0988434195333
epoch: 6, training_loss 1542.8205318482903
epoch: 7, training_loss 1296.6075918182537
epoch: 8, training_loss 1099.8587979779436
epoch: 9, training_loss 937.6252487192874
epoch: 10, training_loss 805.5334117588087
epoch: 11, training_loss 698.4410927951419
epoch: 12, training_loss 611.437348969463
epoch: 13, training_loss 541.4484233296681
epoch: 14, training_loss 482.978456996154
epoch: 15, training_loss 435.24485981858413
epoch: 16, training_loss 395.89561472764257
epoch: 17, training_loss 363.96372707570316
epoch: 18, training_loss 337.2359863629033
epoch: 19, training_loss 314.40385166606364
epoch: 20, training_loss 295.3816715306792
epoch: 21, training_loss 279.3123037045374
epoch: 22, training_loss 265.64869915906246
epoch: 23, tr

### 2.2 Transformer Tagger
    We will also use Transformer to solve the NER problem. You can refer to the following link to implement
    your Transformer tagger: https://pytorch.org/tutorials/beginner/transformer_tutorial.html.

In [18]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [19]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [20]:
# %pip install torchdata

In [21]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# train_iter = WikiText2(split='train')
# type(train_iter)
train_iter = [' '.join(sentence) for sentence, _ in train]
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
# vocab = build_vocab_from_iterator(train_iter, specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [22]:
torch.cuda.is_available()

True

In [23]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator



def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
# train_iter, val_iter, test_iter = WikiText2()
train_iter = [' '.join(sentence) for sentence, _ in train]
val_iter = [' '.join(sentence) for sentence, _ in valid]
test_iter = [' '.join(sentence) for sentence, _ in test]

train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [24]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [25]:
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

In [26]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        if seq_len != bptt:  # only on last batch
            src_mask = src_mask[:seq_len, :seq_len]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            if seq_len != bptt:
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [27]:
best_val_loss = float('inf')
epochs = 150
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

| epoch   1 |   200/  303 batches | lr 5.00 | ms/batch 23.19 | loss  8.03 | ppl  3084.78
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  6.10s | valid loss  6.28 | valid ppl   532.09
-----------------------------------------------------------------------------------------
| epoch   2 |   200/  303 batches | lr 4.75 | ms/batch 10.54 | loss  6.45 | ppl   635.04
-----------------------------------------------------------------------------------------
| end of epoch   2 | time:  3.58s | valid loss  5.87 | valid ppl   355.29
-----------------------------------------------------------------------------------------
| epoch   3 |   200/  303 batches | lr 4.51 | ms/batch 10.61 | loss  5.89 | ppl   362.87
-----------------------------------------------------------------------------------------
| end of epoch   3 | time:  3.60s | valid loss  5.82 | valid ppl   338.51
-------------------------------------------------------------

In [28]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  5.96 | test ppl   388.18


### 2.3 Results
    Please report your best performance on the test set in the same format as the test.txt file, but replace the ground truth labels with your prediction.

In [102]:
def LSTM_calculate_accuracy(model, data, tags, words):
    
    # for s in range(len(data)):
    #     # print(data[s])
    #     for w in range(len(data[s][0])):
    #         # print(data[s][0][w])
    #         if data[s][0][w] not in words:  # word has not been assigned an index yet
    #             data[s][0][w] = '.'

    correct = 0
    # print(tags)

    pred_tags = []
    with torch.no_grad():
        for x, y in data:
            inputs = prepare_sequence(x, words)
            tag_scores = model(inputs)
            # print(x)

            idx = np.argmax(tag_scores, axis=1)
            # print(idx)
            pred_tag = [tags[i] for i in idx]
            # print("predicted result: ", pred_tag)
            # print("ground truth:     ", y)
            pred_tags.append(pred_tag)

            correct += sum([y==pred_tag])
    
    return correct/len(data), pred_tags

In [104]:
# print(tag_to_ix.keys())
tags = np.array([tag for tag in tag_to_ix])

acc_train, pred_train = LSTM_calculate_accuracy(LSTM_model, train, tags, word_to_ix)
print("accuracy of the LSTM model in train data: ", acc_train)

acc_valid, pred_valid = LSTM_calculate_accuracy(LSTM_model, valid, tags, word_to_ix)
print("accuracy of the LSTM model in valid data: ", acc_valid)

acc_test, pred_test = LSTM_calculate_accuracy(LSTM_model, test, tags, word_to_ix)
print("accuracy of the LSTM model in test data: ", acc_test)

accuracy of the LSTM model in train data:  0.958297190898779
accuracy of the LSTM model in valid data:  0.536064627813041
accuracy of the LSTM model in test data:  0.46199782844733983


In [100]:
def LSTM_output_pred(pred, input_file, output_file):
    import os
    import numpy as np

    data_dir = os.path.join(os.getcwd(), "data")
    input_path = os.path.join(data_dir, input_file)
    output_path = os.path.join(data_dir, output_file)

    pred_list = []
    for p in pred:
      for q in p:
        pred_list.append(q)
      pred_list.append('')

    with open(input_path, 'r', encoding="utf-8") as f:
        input = [l.strip().split(' ') for l in f.readlines()]
    f.close()

    # print(len(pred_list), len(input))
    with open(output_path, 'w', encoding="utf-8") as f:
        while len(input) > 0:
          # print(len(pred_list), len(input))
          output = input.pop(0)
          output[-1] = pred_list.pop(0)
          # print(output)
          f.write(' '.join(output) + "\n")
    f.close()

    print(f'Result saved in {output_path} successfully.\n')

In [101]:
LSTM_output_pred(pred_test, 'test.txt', 'LSTM_pred_test.txt')

Result saved in /content/data/LSTM_pred_test.txt successfully.

